In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import os

import pandas as pd
import numpy as np
rng = np.random.default_rng()
import boto3
import joblib

import mtglobals

In [3]:
wage_set = ['0.50','0.90','0.95','0.98','0.99',
            '1.00',
            '1.01','1.02','1.05','1.10','1.50']
#sample = rng.choice(wage_set, 100)

In [4]:
# For testing the distribution
#unique, counts = np.unique(sample, return_counts=True)
#print(np.asarray((unique, counts)).T)

In [5]:
client, mturk_environment = mtglobals.gen_client()
all_hits = mtglobals.download_all_hits(client, start_cutoff=datetime.datetime(2022,1,15,0,0,0))

Your account balance is 499860.00
p2:IaRtYf9ifhGwytMk+QPjgzvQweLYa6oRdhFHMHi2KkAObp/mxrYqZpe3SHK81jc=
p2:zJyVOMKnz6i/v9C9y6NMcIZFv+v5mzNCX8Jkbl3vjQozOjttVa2cW9dNb0HwZYY=
p2:vdMCf9CljFrzAtvgVx2oREc6tKCzeDHe8wG54ZSSAB91vyRegLTZ9bt+jV/g7hE=
p2:oV0WwooKrijuUt3m2y/8imNBHmej846Gl01fiuEzJc3Etxo1o0lbowWcVqAwDto=
p2:lcTV3I4IArKRAcsprnl5B4b0nVl2ITjrpWE70TKSA6J3ILFZZcac/m50GowAgV4=
p2:J+rVQyFJGOyC2c7FDqheBLXAvi9KuitySIfehvJ30OSQVqb5KaFqttXFYolzPeo=
p2:KPbvw7g25wJASBdLiDtACeq9dOFXuiX6gJUgqlzd0062bYTC9obgASoZvtZVitQ=
p2:FqxBqPJ6fo8J9ODTnaqr3NDm70te5jYo85Z3fU8/i4XDd88LVZcC2TOFg4BTMA8=
p2:PkegjnkyesQX4Ppo0QSwYNysV4f8xR3j3Jx5UgRe/l385Dfr31l65/wNhNdDRuGT
p2:axK4LNb8jwEhGE9MU58ajHG5Dq3sbUPUoUGDmnQeLcv9BoNpZsQnAeqTzZylQWV9
p2:RwEc3Sh4IOXv3MphV8fBK6LC/0Cs1MK7yz97ObX85vyfDSZWqgl/7OCJlSJ9ceHi
Creation 2022-01-13 10:14:11-08:00 before start_cutoff


In [6]:
# Filter to get just the stage-1 HITs
stage1_hits = [h for h in all_hits if "instantly unlock" in h['Title']]
[(h['Title'], h['HITId'], h['CreationTime']) for h in stage1_hits]

[('Quick 3-question survey about work [<15 seconds], instantly unlock 2nd-stage HIT with higher reward',
  '3BCRDCM0OEV9AO8YCYWZ0G46CS4K6A',
  datetime.datetime(2022, 1, 31, 19, 10, 51, tzinfo=tzlocal())),
 ('Quick 3-question survey about work [<15 seconds], instantly unlock 2nd-stage HIT with higher reward',
  '3VZYA8PITPZBEU1G3MCCPQBX1Q3051',
  datetime.datetime(2022, 1, 19, 14, 12, 29, tzinfo=tzlocal()))]

In [18]:
# Important: put the HITIds for the stage-1 HIT whose submissions you want to process here
stage1_ids = ['3VZYA8PITPZBEU1G3MCCPQBX1Q3051']

## Load the workers who submitted the stage 1 hit(s) specified above

In [20]:
stage1_hits = [h for h in stage1_hits if h['HITId'] in stage1_ids]
# Now only the specified stage1 hits should remain
#[h['HITId'] for h in stage1_hits]
# Finally, get the list of submitter IDs
all_stage1_submitters = []
for cur_hit in stage1_hits:
    cur_hit_id = cur_hit['HITId']
    cur_submissions = mtglobals.get_hit_submissions(client, cur_hit_id)
    print(len(cur_submissions))
    cur_submitters = [s['WorkerId'] for s in cur_submissions]
    all_stage1_submitters.extend(cur_submitters)

1000


In [21]:
#print(all_stage1_submitters, end='')
len(all_stage1_submitters)

1000

## Functions used in the loop

In [22]:
def check_launched(launched_df, worker_id):
    """
    Check if the 2nd-stage hit has already been launched for this worker. If so,
    return the qual_name, qual_num and offer_amt they were assigned, otherwise return "",-1,-1
    """
    if launched_df is None:
        return "", -1, -1
    result_df = launched_df[launched_df['worker_id'] == worker_id].copy()
    if len(result_df) > 0:
        first_row = result_df.iloc[0]
        qual_name = first_row['qual_name']
        qual_num = int(first_row['qual_num'])
        offer_amt = first_row['offer_amt']
        return qual_name, qual_num, offer_amt
    return "", -1, -1

def gen_custom_hit(cur_worker_id, cur_offer_amt):
    # Now construct the custom HIT for this worker
    # The question we ask the workers is contained in this file.
    # https://github.com/aws-samples/mturk-code-samples/blob/master/Python/my_question.xml
    with open("question_stage2.xml", "r", encoding='utf-8') as f:
        stage2_question = f.read()
    # Fill in the custom data
    stage2_question = stage2_question.replace("!worker_id!",cur_worker_id)
    stage2_question = stage2_question.replace("!offer_amt!",cur_offer_amt)
    return stage2_question

def assign_quals(cur_worker_id, cur_qual_name, cur_qual_id, cur_qual_num, cur_offer_amt):
    # Now assign the custom qualification
    print(f"Associating num {cur_qual_num} with worker {cur_worker_id}")
    response = client.associate_qualification_with_worker(
        # sandbox id
        #QualificationTypeId='3Y0FER3934AFGRF24YM9Q00LF2OXKO',
        # production id, no suffix
        #QualificationTypeId='3QTU8Z4SHN84U5KCFOM3OI2TK3WGK7', 
        # production id, Custom02
        QualificationTypeId=cur_qual_id,
        #WorkerId='AQU32GXNKCCYS', # jpj251@nyu.edu id
        #WorkerId='A3IZUHHAA21KBM', # sn2430 worker sandbox id
        WorkerId=cur_worker_id,
        IntegerValue=cur_qual_num,
        SendNotification=True
    )
    # And update the info in current_qual.txt
    mtglobals.update_current_qual(cur_qual_name, cur_qual_id, cur_qual_num, cur_offer_amt)
    # And finally the participation qual (that everybody gets, regardless of which
    # custom qual)
    response_participation = client.associate_qualification_with_worker(
        # (this is the id for the participation qual)
        QualificationTypeId='303SJT1CWE5D23ZEJCT5ZGDLM1P4FN',
        WorkerId=cur_worker_id,
        IntegerValue=0,
        SendNotification=False
    )
    return response

def gen_qual_restriction(cur_qual_id, cur_qual_num):
    # Example of using qualification to restrict responses to Workers who have had
    # at least 80% of their assignments approved. See:
    # http://docs.aws.amazon.com/AWSMechTurk/latest/AWSMturkAPI/ApiReference_QualificationRequirementDataStructureArticle.html#ApiReference_QualificationType-IDs
    stage2_requirements = [
        #{
        #    'QualificationTypeId': '000000000000000000L0',
        #    'Comparator': 'GreaterThanOrEqualTo',
        #    'IntegerValues': [80],
        #    'RequiredToPreview': True,
        #},
        #{
        #    # WorkplaceSurveyCustom ID in *Sandbox*
        #    #'QualificationTypeId': '3Y0FER3934AFGRF24YM9Q00LF2OXKO',
        #    # WorkplaceSurveyCustom ID in *Dev*
        #    'QualificationTypeId': survey_qual_id,
        #    'Comparator': 'Exists',
        #    'ActionsGuarded': 'DiscoverPreviewAndAccept',
        #},
        {
            'QualificationTypeId': cur_qual_id,
            'Comparator':"In",
            'IntegerValues':[cur_qual_num],
            'ActionsGuarded': 'DiscoverPreviewAndAccept',
        }
    ]
    return stage2_requirements

def launch_custom_hit(cur_worker_id, cur_offer_amt, stage2_question, stage2_requirements):
    # Launch the HIT
    response = client.create_hit(
        # 1 assignment bc single custom stage 2 hit
        MaxAssignments=1,
        LifetimeInSeconds=86400, # 24 hours
        AssignmentDurationInSeconds=3600, # 1 hour
        Reward=cur_offer_amt,
        Title=f'Custom workplace survey HIT for worker id {cur_worker_id}',
        Keywords='survey,workplace,work',
        Description=f'Custom workplace survey HIT for worker id {cur_worker_id}, 30 questions, ~15mins to complete',
        Question=stage2_question,
        QualificationRequirements=stage2_requirements,
    )
    return response

def notify_worker(cur_worker_id, custom_msg):
    response = client.notify_workers(
        Subject='Custom Workplace Survey 2nd-Stage HIT',
        # to send custom_msg
        MessageText=custom_msg,
        # to send payment msg
        #MessageText=payment_msg,
        WorkerIds=[
            cur_worker_id
        ]
    )
    return response

def add_posted_worker(cur_worker_id, cur_offer_amt, cur_qual_name, cur_qual_id,
                      cur_qual_num, custom_url, launched_time):
    # And add to list of already-launched workers
    launched_fpath = mtglobals.stage2_launched_fpath
    if not os.path.isfile(launched_fpath):
        df = pd.DataFrame({'worker_id':[], 'offer_amt':[], 'qual_name':[], 'qual_id':[],
                           'qual_num':[], 'url':[], 'launched_time':[], 'notified_time':[]})
        df.to_csv(launched_fpath, index=False)
    launch_df = pd.read_csv(launched_fpath)
    # The data we're serializing
    new_data = pd.DataFrame({
        'worker_id': [cur_worker_id],
        'offer_amt': [cur_offer_amt],
        'qual_name': [cur_qual_name],
        'qual_id': [cur_qual_id],
        'qual_num': [cur_qual_num],
        'url': [custom_url],
        'launched_time': [launched_time],
        # This one gets filled in by NotifyStage2
        'notified_time': [None],
    })
    updated_df = pd.concat([launch_df, new_data])
    updated_df.to_csv(launched_fpath, index=False)
    return launched_fpath

In [ ]:
# Load the already-launched HITs, if the file exists, for quick checking
if os.path.isfile(mtglobals.stage2_launched_fpath):
    launched_df = pd.read_csv(mtglobals.stage2_launched_fpath)
else:
    launched_df = None
# Loop over workers who submitted stage 1
for worker_num, cur_worker_id in enumerate(all_stage1_submitters):
    print(f"Processing worker #{worker_num}")
    launched_qual_name, launched_qual_num, launched_offer_amt = check_launched(launched_df, cur_worker_id)
    if launched_qual_num != -1:
        print(f"Stage 2 HIT for worker {cur_worker_id} already launched")
        continue
    # If we're here, this worker_id hasn't had a stage 2 HIT launched yet
    cur_qual_info = mtglobals.get_current_qual()
    cur_qual_name = cur_qual_info['qual_name']
    cur_qual_id = cur_qual_info['qual_id']
    last_qual_num = cur_qual_info['last_qual_num']
    last_offer = cur_qual_info['last_offer_amt']
    print(f"Last used qual num: {last_qual_num}")
    print(f"Last used offer amt: {last_offer}")
    # Generate random offer amount
    # New new: randomly sampled
    cur_offer_amt = rng.choice(wage_set)
    cur_qual_num = (last_qual_num + 1) % 100
    print(f"Processing worker {cur_worker_id}, offer {cur_offer_amt}, qual_num {cur_qual_num}")
    # Using the functions to create the HIT params and content
    # Generate the xml for the question
    stage2_question = gen_custom_hit(cur_worker_id, cur_offer_amt)
    # Assign the custom qualification num for this worker
    qual_response = assign_quals(cur_worker_id, cur_qual_name, cur_qual_id, cur_qual_num, cur_offer_amt)
    print("Qualification assignment response:")
    print(qual_response)
    # Generate the data structure which enforces the qualification restriction
    stage2_requirements = gen_qual_restriction(cur_qual_id, cur_qual_num)
    # And launch the HIT
    launch_response = launch_custom_hit(cur_worker_id, cur_offer_amt, stage2_question, stage2_requirements)
    print("Launch response:")
    print(launch_response['ResponseMetadata']['HTTPStatusCode'])
    launched_time = launch_response['HIT']['CreationTime']
    
    # The response included several fields that will be helpful later
    hit_type_id = launch_response['HIT']['HITTypeId']
    hit_id = launch_response['HIT']['HITId']
    #print("Created HITTypeId: {}".format(hit_type_id))
    print(f"Created HITId {hit_id} for worker {cur_worker_id}")
    custom_url = mturk_environment['preview'] + "?groupId={}".format(hit_type_id)
    print(f"You can work the HIT here: {custom_url}")
    #print("\nAnd see results here:")
    #print(mturk_environment['manage'])
    mtglobals.write_log(f"Created custom stage-2 HIT for worker {cur_worker_id}, offer {cur_offer_amt}, URL {custom_url}")
    # And append the data for this iteration to the .csv of launched workers
    launched_fpath = add_posted_worker(cur_worker_id, cur_offer_amt, cur_qual_name, cur_qual_id,
                                       cur_qual_num, custom_url, launched_time)
    print(f"New launched HIT successfully added to {launched_fpath}")
    print("=====[ end loop iteration ]=====")


In [ ]:
# 7:16pm, 12/8/2021

In [13]:
# 1:59pm, 12/1/2021

In [14]:
# 11:18am, 11/24/2021

In [15]:
# 10:52am, 11/20/2021

In [16]:
# 2:18pm, 10/30/2021